# Import Packages

In [30]:
import os
import pandas as pd

# Load Match Data

In [31]:
# Load the CSV file into a DataFrame
matches_csv_path = "matches_2.csv"
matches_df = pd.read_csv(matches_csv_path)

# Display the DataFrame
matches_df.head()

<ipython-input-31-574c9cf74218>:3: DtypeWarning: Columns (118,119,121,122,124,126,128,129,131,133,135,136,138,139,141,142,144,146,148,149,151,153,155,156) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_df = pd.read_csv(matches_csv_path)


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,home_team_losses,away_team_losses,home_team_formation,away_team_formation,home_team_num_defenders,home_team_num_midfielders,home_team_num_attackers,away_team_num_defenders,away_team_num_midfielders,away_team_num_attackers
0,4778,4769,4769,2008/2009,1,2008-08-09,483138,9873,9853,1,...,0.0,0.0,4-4-2,4-5-1,4,4,2,4,5,1
1,4777,4769,4769,2008/2009,1,2008-08-09,483137,9874,9855,1,...,0.0,0.0,4-4-2,4-4-2,4,4,2,4,4,2
2,4776,4769,4769,2008/2009,1,2008-08-09,483136,9851,8592,4,...,0.0,0.0,4-4-2,4-4-2,4,4,2,4,4,2
3,4775,4769,4769,2008/2009,1,2008-08-09,483135,8481,8639,0,...,0.0,0.0,4-5-1,4-3-3,4,5,1,4,3,3
4,4774,4769,4769,2008/2009,1,2008-08-09,483134,9829,9847,1,...,0.0,0.0,4-4-2,4-4-2,4,4,2,4,4,2


In [24]:
matches_df.shape

(16637, 1072)

# Data Cleaning

In [27]:
matches_df["home_player_X1"]

,home_player_X1
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
16632,1.0
16633,1.0
16634,1.0
16635,1.0


In [32]:
def process_match_data(df):
    """
    Preprocess the matches DataFrame by performing the following steps:
    1. Drop specific in-game event columns and rank-related columns.
    2. Drop columns with more than 400 missing values.
    3. Drop rows with any missing values.
    4. Remove rows with invalid formations for home and away teams.

    Input: df (pd.DataFrame): The input DataFrame to preprocess.

    Output: pd.DataFrame: The preprocessed DataFrame.
    """

    # Drop player position columns since we already have frmation and do not need them
    for i in range(11):

      df.drop(columns = [f"home_player_X{i+1}"], inplace = True)
      df.drop(columns = [f"away_player_X{i+1}"], inplace = True)
      df.drop(columns = [f"home_player_Y{i+1}"], inplace = True)
      df.drop(columns = [f"away_player_Y{i+1}"], inplace = True)

    # add Year and Month columns since we will drop date
    df["date"] = pd.to_datetime(df["date"])
    df["Year"] = df["date"].dt.year
    df["Month"] = df["date"].dt.month

    # Drop in-game events and rank-related columns
    columns_to_drop = [
        "id", "home_team_goal", "away_team_goal", "date", "goal_difference", "date"
    ]
    df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    # Drop columns with more than 400 missing values
    df.drop(columns=[col for col in df.columns if df[col].isnull().sum() > 400], inplace=True)

    # Drop rows with any missing values
    df.dropna(inplace=True)

    # Define invalid formations for home and away teams
    invalid_home_formations = ['1-1-1-1-1-1-1-1-1-1', '6-3', '4-2-2-1']
    invalid_away_formations = ['1-1-1-1-1-1-1-1-1-1', '2-3-1-1', '3-3-1-1-1', '3-2-1-1']

    # Remove rows with invalid formations
    df = df[~df["home_team_formation"].isin(invalid_home_formations)]
    df = df[~df["away_team_formation"].isin(invalid_away_formations)]

    return df

# Apply the function to matches_df
matches_df = process_match_data(matches_df)

# Display the result
matches_df

,country_id,league_id,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_player_1,home_player_2,home_player_3,...,home_team_formation,away_team_formation,home_team_num_defenders,home_team_num_midfielders,home_team_num_attackers,away_team_num_defenders,away_team_num_midfielders,away_team_num_attackers,Year,Month
3,4769,4769,2008/2009,1,483135,8481,8639,26233.0,111909.0,31684.0,...,4-5-1,4-3-3,4,5,1,4,3,3,2008,8
4,4769,4769,2008/2009,1,483134,9829,9847,145039.0,41097.0,26216.0,...,4-4-2,4-4-2,4,4,2,4,4,2,2008,8
9,4769,4769,2008/2009,1,483133,9748,9941,26295.0,26162.0,37553.0,...,4-3-3,4-3-3,4,3,3,4,3,3,2008,8
10,7809,7809,2008/2009,1,499317,9823,9790,27284.0,35988.0,39774.0,...,4-1-2-1-2,4-3-3,4,4,2,4,3,3,2008,8
20,4769,4769,2008/2009,2,483146,9847,9827,30742.0,20482.0,94043.0,...,4-5-1,4-4-2,4,5,1,4,4,2,2008,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16630,10257,10257,2015/2016,38,2060636,10233,8524,162885.0,164029.0,30928.0,...,3-5-2,4-2-3-1,3,5,2,4,5,1,2016,5
16631,10257,10257,2015/2016,38,2060637,8533,9857,150464.0,39273.0,24504.0,...,4-3-1-2,4-2-3-1,4,4,2,4,5,1,2016,5
16632,1729,1729,2015/2016,38,1987606,8659,8650,24147.0,169162.0,32441.0,...,4-2-3-1,4-3-3,4,5,1,4,3,3,2016,5
16633,10257,10257,2015/2016,38,2060639,8543,8535,39599.0,39562.0,40016.0,...,4-3-3,3-4-2-1,4,3,3,3,6,1,2016,5


# Save to CSV

In [33]:
# Create the directory
output_dir = "Data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the matches_df DataFrame to a CSV file
output_path = os.path.join(output_dir, "matches_3.csv")
matches_df.to_csv(output_path, index=False)

print(f"matches_df has been saved to {output_path}")

matches_df has been saved to Data/matches_3.csv
